In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from collections import deque
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [2]:
np.random.seed(10)

data = []
row_ind = []
col_ind = []

max_c = 0

with open('train.dat') as f:
    for i, line in enumerate(f):
        r = np.array(line.split(' '))
        r = r.reshape((int(len(r)/2), 2))
        for j, k in r:
            row_ind.append(i)
            col_ind.append(int(j))
            max_c = max(max_c, int(j))
            data.append(int(k))
print(max_c)
mat = csr_matrix((np.array(data), (np.array(row_ind), np.array(col_ind))) , shape=(8580, max_c+1))

126355


In [ ]:
tfidf = TfidfTransformer()
tfidf_mat = tfidf.fit_transform(mat)
pca = PCA(n_components=12)
tfidf_mat_pca = pca.fit_transform(tfidf_mat.toarray())

In [ ]:
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
kmeans = KMeans(n_clusters = 200, max_iter = 300, verbose=1)

In [ ]:
kmeans.fit(tfidf_mat_pca)
centers = kmeans.cluster_centers_

In [ ]:
centers = kmeans.cluster_centers_
# for center in centers:
#     print(np.linalg.norm(center))
centers_sparse = csr_matrix(centers)
sims_t = centers_sparse.dot(centers_sparse.T).toarray()
# sims_t[1,2]

In [ ]:
def cossim(v1, v2):
    return sims_t[v1,v2] / ((sims_t[v1,v1]*sims_t[v2,v2])**0.5)

In [ ]:
sims_matrix = np.zeros((len(centers), len(centers)), dtype='float32')
# sims_matrix = cosine_similarity(centers_sparse, centers_sparse)



In [ ]:
for i in range(len(centers)):
    for j in range(i, len(centers)):
#         print(i, j)
        if i == j:
            sims_matrix[i, j] = 1.0
            continue
        sims_matrix[i,j] = cossim(i, j)
        sims_matrix[j,i] = sims_matrix[i,j]
sims_matrix


In [ ]:
def dbscan(sims, eps=0.5, minpts=10):
    graph = np.ndarray(shape=(sims.shape[0],), dtype=object)
    label = [0]*tfidf_mat.shape[0]
    kernels = []
    for i in range(sims.shape[0]):
        cns = [j for j in range(sims.shape[0]) if sims[i, j] > eps and not i == j]
        graph[i] = np.array(cns)
        if len(cns) > minpts:
            label[i] = 2
            kernels.append(i)



    for kernel in kernels:
        for pt in graph[kernel]:
            if label[pt] < 2:
                label[pt] = 1

    conncomps = []
    all_set = set([x for x in range(len(label)) if not label[x] == 0])
    while all_set:
        a = all_set.pop()
        conncomp = []
        aqueue = deque([a])
        while aqueue:
            n = aqueue.pop()
            for m in graph[n]:
                if m in all_set:
                    all_set.remove(m)
                    aqueue.append(m)
                    conncomp.append(m)
        conncomps.append(conncomp)

    # len(conncomps), conncomps

    dl = np.array([-1]*len(graph))

    for i in range(len(conncomps)):
        for index in conncomps[i]:
    #         print(index)
            dl[index] = i
    print(len(conncomps))
    return dl

In [ ]:
# for i in dl:
#     if not i == -1:
#         print(i)
# dl
for i in range(1):
    dl = dbscan(sims_matrix, 0.85, 5)
    print(i, dl)

In [ ]:
# np.count_nonzero(dl == -1)/len(dl), np.count_nonzero(dl == 0)/len(dl)


In [ ]:
# silhouette_score(cosine_similarity(tfidf_mat, tfidf_mat), dl)

In [ ]:
labels = kmeans.labels_
labels.shape

In [ ]:
ans = []
for l in labels:
    ans.append(dl[l]+1)

# ans.__len__(), ans

In [ ]:
# dist_mat = tfidf_mat.dot(tfidf_mat.T).toarray()
# dist_mat.shape

In [ ]:
# # np.diagonal(dist_mat.reshape((8580, 8580,)))
# for i in range(len(dist_mat)):
#     for j in range(len(dist_mat[i])):
# #         print(i,j)
#         dist_mat[i,j] = 1-dist_mat[i,j]

In [ ]:
dist_mat

In [ ]:
# silhouette_score(tfidf_mat.toarray(), ans, metric='cosine')

In [ ]:
with open('res9.dat', 'w') as f:
    for a in ans:
        f.write(str(a)+'\n')